In [1]:
## Basic import and DF creation

import pandas as pd

# Specify the path to your Excel file
file_path = r"09_Outputs\main_data.xlsx"

# Load the Excel file into a DataFrame using the openpyxl engine
df_main = pd.read_excel(file_path, engine='openpyxl')

# Remove quotation marks from the 'Entity' column
df_main['Entity'] = df_main['Entity'].str.replace('"', '')

# Retrieve unique countries from the DataFrame and store them as a list of strings
all_countries = df_main['Entity'].drop_duplicates().astype(str).tolist()

# Display the first 5 rows of the DataFrame for a quick overview
df_main.head()


,Entity,Year,Code,Annual CO₂ emissions in tons,Renewables - TWh growth - equivalent,Annual CO₂ emissions from flaring,Annual CO₂ emissions from gas,Annual CO₂ emissions from oil,Annual CO₂ emissions from coal,Primary energy consumption (TWh),...,Other renewables excluding bioenergy (TWh),Oil consumption - TWh,Primary energy consumption per GDP (kWh/$),Low-carbon electricity (TWh),GDP (constant 2015 US$),Renewable electricity (% electricity production) (World Bank (2015)),Solar photovoltaic capacity,Annual greenhouse gas emissions in CO2 equivalents,Continent,Electricity from wind (TWh)
0,Afghanistan,1965,AFG,1006917.0,NaN,0.0,0.0,542272.0,381056.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24430504.0,NaN,NaN
1,Afghanistan,1966,AFG,1091159.0,NaN,0.0,0.0,575248.0,428688.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25174948.0,NaN,NaN
2,Afghanistan,1967,AFG,1281865.0,NaN,0.0,260144.0,556928.0,399376.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24644464.0,NaN,NaN
3,Afghanistan,1968,AFG,1223391.0,NaN,0.0,347041.0,496817.0,332429.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24382198.0,NaN,NaN
4,Afghanistan,1969,AFG,941232.0,NaN,0.0,0.0,527616.0,362736.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23233260.0,NaN,NaN


In [2]:
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
from IPython.core.display import HTML

# Presuming df_main is already available
filtered_df = df_main

# Extract unique countries from the dataframe
all_countries = sorted(filtered_df['Entity'].drop_duplicates().astype(str).tolist())
default_countries = ['Germany', 'China', 'India', 'United States', 'World']
left_container_width = '270px'

def create_interface_for_column(column):
    # Local functions to keep independence of widgets
    def update_country_list(*args):
        selected_countries = [cb.description for cb in local_checkboxes_container.children if cb.value]
        search_input = local_search_box.value
        search_results = [country for country in all_countries if search_input.lower() in country.lower()]
        updated_countries = list(set(search_results + selected_countries))
        updated_countries = sorted(updated_countries, key=lambda x: (x not in selected_countries, x))
        updated_checkboxes = [widgets.Checkbox(value=(country in selected_countries), description=country,
                                               layout=widgets.Layout(width="100%", padding="0px")) for country in updated_countries]
        for cb in updated_checkboxes:
            cb.observe(plot_for_column, names='value')
        local_checkboxes_container.children = tuple(updated_checkboxes)
        plot_for_column()  # Call plot function every time checkboxes are updated

    def plot_for_column(change=None):
        selected_countries = [cb.description for cb in local_checkboxes_container.children if cb.value]
        sorted_checkboxes = sorted(local_checkboxes_container.children, key=lambda cb: (cb.description not in selected_countries, cb.description))
        local_checkboxes_container.children = tuple(sorted_checkboxes)
        with local_out:
            clear_output(wait=True)
            fig = go.Figure()
            fig_height = 400  
            fig.update_layout(height=fig_height)
            for country in selected_countries:
                country_data = filtered_df[filtered_df['Entity'] == country]
                fig.add_trace(go.Scatter(x=country_data['Year'], y=country_data[column], mode='lines', name=country, showlegend=True))
            fig.update_layout(title=column, xaxis_title='Year', yaxis_title=column, plot_bgcolor='white', paper_bgcolor='white')
            fig.show()

    # Local search box and checkboxes for each column
    local_search_box = widgets.Text(value='', placeholder='Search for Countries...', description='', layout=widgets.Layout(width="95%"))
    local_search_box.observe(update_country_list, names='value')
    local_clear_button = widgets.Button(description="X", layout=widgets.Layout(width="5%"))
    local_clear_button.on_click(lambda change: setattr(local_search_box, 'value', ''))
    
    local_country_checkboxes = [widgets.Checkbox(value=(country in default_countries), description=country,
                                                 layout=widgets.Layout(width="100%", padding="0px")) for country in all_countries]
    local_checkboxes_container = widgets.VBox(local_country_checkboxes, layout=widgets.Layout(overflow_y="scroll", height="400px", border="1px solid #DDD"))

    local_left_container = widgets.VBox([widgets.HBox([local_search_box, local_clear_button]), local_checkboxes_container],
                                        layout=widgets.Layout(width=left_container_width, border="1px solid #DDD", padding="5px 0px 0px 0px"))

    local_out = widgets.Output(layout=widgets.Layout(height="400px", width="60%"))
    
    # Displaying the UI for the column
    display(Markdown(f"## Diagramm für {column}"))
    display(widgets.HBox([local_left_container, local_out]))
    plot_for_column()

# Für jede Spalte, die Sie plotten möchten:
columns_to_plot = filtered_df.columns[filtered_df.columns.get_loc('Code')+1:]
for col in columns_to_plot:
    create_interface_for_column(col)

display(HTML("""
<style>
    :root {
        --jp-widgets-inline-label-width: auto !important;
    }
</style>
"""))


## Diagramm für Annual CO₂ emissions in tons

## Diagramm für Renewables - TWh growth - equivalent

## Diagramm für Annual CO₂ emissions from flaring

## Diagramm für Annual CO₂ emissions from gas

## Diagramm für Annual CO₂ emissions from oil

...

## Diagramm für Renewable electricity (% electricity production) (World Bank (2015))

## Diagramm für Solar photovoltaic capacity

## Diagramm für Annual greenhouse gas emissions in CO2 equivalents

## Diagramm für Continent

## Diagramm für Electricity from wind (TWh)